# Soldering pen design

Initialization of iPython, some helper functions.

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def readCsvData(fileName):
    return np.genfromtxt(fileName, delimiter=';')

def plotData(data, timebase=1, xlabel="", ylabel="",names=[]):
    fig=plt.figure(figsize=[10,5])
    l = np.size(data,0)
    time = np.linspace(0,l-1,l)*timebase
    if np.ndim(data)>1: # If more than one vector to plot
        count = np.size(data,1)
        for i in range(count):
            plt.plot(time, data[:,i], label=names[i])
    else:
        plt.plot(time, data, label=names[0])
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.plot()
    
def plotCsv(fileName, timebase=1, xlabel="", ylabel="",names=[]):
    data = readCsvData(fileName)
    plotData(data, timebase, xlabel, ylabel,names)

## Magnetometer

Permanent neodymium magnet is mounted in soldering iron stand. On board MEMS magnetometer is used to sense its' proximity. If magnet is detected, soldering iron goes to sleep mode after programmed time.
MAG3110 has full scale range of $\pm 1000 \mu T$. Zero flux offset is typically $\pm 100 \mu T$. It's large, compared to typical geomagnetic field strength of $25\ldots60 \mu T$. Zero offset calibration is desirable.

### Magnitude calculation
For calculating magnitude of magnetic field flux density from X, Y, Z readings, following formula should be used:

$$m=\sqrt{x^2+y^2+z^2}$$

To minimize calculations, approximation will be used. Derivative of alpha max plus beta min algorithm will be used:

$$m_{appr}=a*abs(x)+b*abs(y)+c*abs(z)$$

Simple values for a, b, c are selected:

$$m_{appr}=1*abs(x)+\frac{1}{2}*abs(y)+\frac{1}{4}*abs(z)$$

Let's calculate error of this formula.

In [6]:
a=np.linspace(0.01,1,100) # max parameter change
data=[[[[x,y,z] for x in a] for y in a] for z in a] # generate all possible triplets
mag=[[[np.sqrt(x*x+y*y+z*z) for x in a] for y in a] for z in a] # generate magnitude for all possible triplets
# generate magnitude approximation for all possible triplets
mag_ap=[[[(np.max([x,y,z]) + np.median([x,y,z])/2 + np.min([x,y,z])/4) for x in a] for y in a] for z in a] 
# calculate error
err=np.divide(mag_ap,mag)
print("Max error: ",np.max(err),", min error:",np.min(err))

Max error:  1.14564392374 , min error: 1.00739926511


15% error is perfectly acceptable. It's even better, because error is always >=0, so relatively it's +-7.5%.